In [51]:
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as Imputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)

In [52]:
import joblib


In [120]:
train = pd.read_csv('train.csv')

In [121]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2275 non-null   int64  
 1   District       2275 non-null   object 
 2   Season         2275 non-null   object 
 3   Year           2275 non-null   int64  
 4   Crop           2275 non-null   object 
 5   AvgAirTemp     2256 non-null   float64
 6   MinAirTemp     2274 non-null   float64
 7   MaxAirTemp     2274 non-null   float64
 8   AvgTempSkew    2251 non-null   float64
 9   AvgTempKurt    2251 non-null   float64
 10  AvgRelHum      2256 non-null   float64
 11  AvgRelHumSkew  2251 non-null   float64
 12  AvgRelHumKurt  2251 non-null   float64
 13  AvgDewPt       2256 non-null   float64
 14  AvgDewPtSkew   2251 non-null   float64
 15  AvgDewPtKurt   2251 non-null   float64
 16  AvgPrec        1350 non-null   float64
 17  AvgPrecSkew    1347 non-null   float64
 18  AvgPrecK

In [55]:
train.columns

Index(['ID', 'District', 'Season', 'Year', 'Crop', 'AvgAirTemp', 'MinAirTemp',
       'MaxAirTemp', 'AvgTempSkew', 'AvgTempKurt', 'AvgRelHum',
       'AvgRelHumSkew', 'AvgRelHumKurt', 'AvgDewPt', 'AvgDewPtSkew',
       'AvgDewPtKurt', 'AvgPrec', 'AvgPrecSkew', 'AvgPrecKurt', 'AvgWind',
       'Yield'],
      dtype='object')

# NaN 

In [122]:
train.isnull().sum().sort_values(ascending=False).head()

AvgPrecSkew      928
AvgPrec          925
AvgPrecKurt      233
AvgRelHumKurt     24
AvgTempKurt       24
dtype: int64

In [56]:
train = train[['Season', 'Crop', 'AvgAirTemp', 'MinAirTemp',
       'MaxAirTemp', 'AvgTempSkew', 'AvgTempKurt', 'AvgRelHum',
       'AvgRelHumSkew', 'AvgRelHumKurt', 'AvgDewPt', 'AvgDewPtSkew',
       'AvgDewPtKurt', 'AvgPrec', 'AvgPrecSkew', 'AvgPrecKurt', 'AvgWind',
       'Yield']]

In [58]:
le = preprocessing.LabelEncoder()

In [59]:
le.fit(train.Crop)

LabelEncoder()

In [60]:
train.Crop = le.transform(train.Crop)

/usr/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [61]:
le_season = preprocessing.LabelEncoder()
le_season.fit(train.Season)
train.Season = le_season.transform(train.Season)

In [62]:
train

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind,Yield
0,0,48,25.715217,19.8,33.9,0.099351,-0.671190,72.956522,-0.039249,-0.795628,20.498913,0.119008,0.208604,11.509783,3.768110,15.894109,4.764130,0.540000
1,1,16,24.581111,17.4,34.8,1.721024,2.574109,78.013333,-1.873517,3.108912,20.342222,-2.177090,4.973371,8.950000,4.966012,2.091170,3.876667,0.890000
2,1,39,26.710995,12.2,43.6,0.534709,-0.436029,49.956195,0.661461,-0.947307,14.037696,0.052482,-0.697203,1.510646,8.818788,106.921134,2.821990,0.320000
3,1,54,16.551429,7.5,26.1,-0.039599,-1.254013,74.351429,-0.840153,-0.349229,11.874286,0.293178,-0.822140,1.197143,7.093744,2.823910,2.388571,2.370000
4,3,17,26.084384,10.9,40.2,-0.193347,-0.242647,54.783562,-0.054892,-1.419339,14.782466,-0.360214,-0.990542,NaN,NaN,-0.990542,2.915616,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,2,23,28.716393,16.1,41.5,0.009805,-0.848559,48.536885,0.402157,-1.288343,15.421311,-0.216475,-1.017080,1.417213,3.891426,-1.017080,2.717213,3.060000
2271,3,0,26.145879,15.2,40.6,0.537010,-0.403752,56.954670,-0.013288,-1.381390,15.984615,-0.812412,-0.029147,3.211944,11.257709,3.279350,3.350275,1.400000
2272,0,46,31.734959,21.4,43.2,-0.067425,-0.703497,47.711382,0.520505,-0.935284,18.027642,-0.354586,-0.759088,NaN,NaN,-0.759088,3.519512,0.760000
2273,1,15,24.505921,11.9,40.8,0.008320,-0.811582,48.948684,-0.156759,-0.453292,12.450000,-0.105224,-0.628325,0.135526,6.921572,-0.628325,2.790132,0.568182


In [63]:
mean = train.groupby('Crop').apply(lambda g: g.mean(skipna=True))


In [64]:
mean

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind,Yield
Crop,,,,,,,,,,,,,,,,,,
0,3.000000,0.0,25.927966,17.544444,36.238889,0.269202,0.004603,68.586467,-0.514699,-0.533040,19.245720,-0.840079,-0.063003,3.227587,10.334409,13.628211,4.223316,3.357222
1,0.118812,1.0,25.667408,17.106931,36.291089,0.517692,0.216889,66.363630,-0.540473,-0.007814,18.209499,-0.954599,1.241992,4.811188,6.551720,4.281267,3.519411,0.740651
2,0.095238,2.0,26.705640,18.030952,37.283333,0.557317,0.383899,67.967853,-0.665539,-0.002227,19.570170,-1.158555,1.733777,5.177749,4.367523,3.846829,3.660105,1.108037
3,3.000000,3.0,22.977231,11.351852,36.933333,0.254502,-0.221808,64.146325,-0.342461,-0.527962,14.958516,-0.600899,0.055270,3.569018,7.367241,4.570192,3.576036,23.446296
4,1.000000,4.0,24.472123,8.004878,40.202439,0.117801,-0.331414,53.623583,0.091438,-0.170709,13.148814,-0.391009,0.072874,2.485778,4.405056,3.670913,2.542468,2.385650
5,3.000000,5.0,26.282865,19.920000,33.093333,-0.122240,-0.056450,73.890965,-0.679052,0.058583,21.177919,-0.960692,0.657506,3.364670,4.544113,18.289596,4.774763,1.928000
6,3.000000,6.0,24.637088,14.440000,36.260000,0.044330,0.174318,73.800385,-0.739592,0.123228,19.476593,-0.944105,0.275752,4.394579,3.327321,13.435172,3.259231,0.080000
7,3.000000,7.0,24.983022,15.060000,36.480000,0.171532,0.357664,71.730165,-0.652254,-0.147711,19.261538,-1.100571,0.769642,4.296637,3.001411,10.243379,3.705769,0.550000
8,0.050847,8.0,26.573401,17.927119,36.788136,0.674504,0.549849,70.617749,-0.742006,0.075234,20.243978,-1.213880,1.835746,6.275060,3.040326,3.573958,3.737564,0.871514


In [65]:
train = train.fillna(train.groupby('Crop').transform('mean'))


In [66]:
imp = Imputer(missing_values=np.nan, strategy='mean')
imp.fit(train)
train = pd.DataFrame(data=imp.transform(train), columns=train.columns)

In [67]:
train

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind,Yield
0,0.0,48.0,25.715217,19.8,33.9,0.099351,-0.671190,72.956522,-0.039249,-0.795628,20.498913,0.119008,0.208604,11.509783,3.768110,15.894109,4.764130,0.540000
1,1.0,16.0,24.581111,17.4,34.8,1.721024,2.574109,78.013333,-1.873517,3.108912,20.342222,-2.177090,4.973371,8.950000,4.966012,2.091170,3.876667,0.890000
2,1.0,39.0,26.710995,12.2,43.6,0.534709,-0.436029,49.956195,0.661461,-0.947307,14.037696,0.052482,-0.697203,1.510646,8.818788,106.921134,2.821990,0.320000
3,1.0,54.0,16.551429,7.5,26.1,-0.039599,-1.254013,74.351429,-0.840153,-0.349229,11.874286,0.293178,-0.822140,1.197143,7.093744,2.823910,2.388571,2.370000
4,3.0,17.0,26.084384,10.9,40.2,-0.193347,-0.242647,54.783562,-0.054892,-1.419339,14.782466,-0.360214,-0.990542,2.177847,6.846041,-0.990542,2.915616,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,2.0,23.0,28.716393,16.1,41.5,0.009805,-0.848559,48.536885,0.402157,-1.288343,15.421311,-0.216475,-1.017080,1.417213,3.891426,-1.017080,2.717213,3.060000
2271,3.0,0.0,26.145879,15.2,40.6,0.537010,-0.403752,56.954670,-0.013288,-1.381390,15.984615,-0.812412,-0.029147,3.211944,11.257709,3.279350,3.350275,1.400000
2272,0.0,46.0,31.734959,21.4,43.2,-0.067425,-0.703497,47.711382,0.520505,-0.935284,18.027642,-0.354586,-0.759088,6.434152,4.970937,-0.759088,3.519512,0.760000
2273,1.0,15.0,24.505921,11.9,40.8,0.008320,-0.811582,48.948684,-0.156759,-0.453292,12.450000,-0.105224,-0.628325,0.135526,6.921572,-0.628325,2.790132,0.568182


In [68]:
train = train.values

In [69]:
train.shape

(2275, 18)

In [70]:
X = train[:,0:-1]
y = train[:,-1]
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [29]:
hidden = 512

In [71]:
x_scaler = MinMaxScaler()
x_scaler = x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(y_train.reshape(-1, 1))
y_train = y_scaler.transform(y_train.reshape(-1, 1))


In [72]:
X_validation = x_scaler.transform(X_validation)
y_validation = y_scaler.transform(y_validation.reshape(-1, 1))

In [73]:
joblib.dump(x_scaler, 'x_scaler.pkl')
joblib.dump(y_scaler,'y_scaler.pkl')

['y_scaler.pkl']

# NEURAL NETWORK

In [30]:

model = Sequential()
model.add(BatchNormalization())
model.add(Dense(units=hidden, activation='relu', input_dim=X_train.shape[1], kernel_initializer='glorot_normal'))
model.add(Dense(units=hidden // 2, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dense(units=hidden // 4, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dense(units=hidden // 8, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dense(units=1))

In [31]:
check_pointer = ModelCheckpoint(filepath='crops.hdf5', verbose=0, save_best_only=True,
                                        save_weights_only=False)
model.compile(optimizer="Nadam", loss="mean_squared_error", metrics=["mean_squared_error"])

early_stopping = EarlyStopping(monitor="val_mean_squared_error", patience=5, verbose=1,
                               mode='auto', restore_best_weights=True)
model.fit(X_train, y_train, epochs=25, batch_size=256,
          validation_data=(X_validation, y_validation), callbacks=[early_stopping, check_pointer])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 1592 samples, validate on 683 samples
Epoch 1/25
1592/1592 [==============================] - 0s 259us/step - loss: 0.0564 - mean_squared_error: 0.0564 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 2/25
1592/1592 [==============================] - 0s 35us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 3/25
1592/1592 [==============================] - 0s 41us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0015 - val_mean_squared_error: 0.0015
Epoch 4/25
1592/1592 [==============================] - 0s 38us/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 5/25
1592/1592 [==============================] - 0s 40us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 6/25
1592/1592 [=======================

# Random Forest REGRESSOR

In [74]:
from sklearn.ensemble import RandomForestRegressor

In [75]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [76]:
regr.fit(X_train, y_train)

<ipython-input-76-54b5df3622f8>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [77]:
y_pred=regr.predict(X_validation)

In [78]:
from sklearn.metrics import mean_squared_error

In [79]:
mean_squared_error(y_validation, y_pred)

0.0009092611274203688

In [80]:
filename = 'finalized_model.sav'
joblib.dump(regr, filename)

['finalized_model.sav']

# TEST DATA

In [81]:
test = pd.read_csv('test.csv')

In [82]:
test

,ID,District,Season,Year,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind
0,2492,Jaipur,Rabi,2003,Barley,28.131707,19.3,41.0,1.063315,0.476075,69.618699,-0.852685,-0.666886,21.366667,-1.674645,3.143399,NaN,NaN,3.143399,2.956911
1,1061,Coimbatore,Kharif,1999,Maize,25.703947,20.0,36.9,1.181141,1.730647,66.792105,-1.093754,1.552880,18.850000,-2.437715,8.002811,6.980263,2.197078,4.908670,4.723684
2,2422,Dharwad,Kharif,2005,Niger seed,24.614444,18.9,34.1,0.310620,-0.477274,80.448889,-0.873560,0.647786,21.021111,-0.342050,1.124286,6.210000,9.549663,2.919901,4.145556
3,175,Dharwad,Kharif,2011,Ragi,25.745556,20.7,41.5,0.538028,-0.982833,77.014444,-1.034927,-0.239949,21.252222,-0.910521,1.303033,NaN,NaN,NaN,4.690000
4,1928,Indore,Whole year,2007,Sesamum,26.042033,8.2,43.5,-0.024875,-0.292127,49.420879,-1.300360,0.493824,13.131044,0.027065,-1.316498,2.143956,77.109252,7.377355,2.607967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,690,Jalna,Kharif,2003,Moong,24.221739,17.0,29.9,0.060319,-0.178136,85.498913,-1.928440,4.796944,21.627174,-1.389769,2.260434,6.247826,4.602014,27.896006,3.513043
565,1377,Dharwad,Whole year,2009,Sannhamp,25.357143,14.4,38.7,0.754646,0.289155,65.911264,-0.283801,-1.338683,18.016209,-1.063751,0.419328,2.120330,33.080004,4.866351,3.153571
566,2376,Coimbatore,Kharif,2001,Ragi,25.138816,15.6,34.8,1.004186,1.678955,72.937500,-0.931120,0.947053,19.778947,-1.977677,5.674409,6.615132,2.971615,9.914768,4.592105
567,1332,Tonk,Kharif,1998,Soyabean,29.697561,19.6,42.1,0.298462,-1.048559,58.222764,0.036185,-1.381920,19.526016,-1.020817,1.238418,NaN,NaN,1.238418,3.490244


In [83]:
test.Crop = le.transform(test.Crop)

In [84]:
id_crop = test.ID.values

In [85]:
test.Season = le_season.transform(test.Season)

In [86]:
test =  test[['Season', 'Crop', 'AvgAirTemp', 'MinAirTemp',
       'MaxAirTemp', 'AvgTempSkew', 'AvgTempKurt', 'AvgRelHum',
       'AvgRelHumSkew', 'AvgRelHumKurt', 'AvgDewPt', 'AvgDewPtSkew',
       'AvgDewPtKurt', 'AvgPrec', 'AvgPrecSkew', 'AvgPrecKurt', 'AvgWind']]

In [87]:
test

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind
0,1,4,28.131707,19.3,41.0,1.063315,0.476075,69.618699,-0.852685,-0.666886,21.366667,-1.674645,3.143399,NaN,NaN,3.143399,2.956911
1,0,23,25.703947,20.0,36.9,1.181141,1.730647,66.792105,-1.093754,1.552880,18.850000,-2.437715,8.002811,6.980263,2.197078,4.908670,4.723684
2,0,29,24.614444,18.9,34.1,0.310620,-0.477274,80.448889,-0.873560,0.647786,21.021111,-0.342050,1.124286,6.210000,9.549663,2.919901,4.145556
3,0,38,25.745556,20.7,41.5,0.538028,-0.982833,77.014444,-1.034927,-0.239949,21.252222,-0.910521,1.303033,NaN,NaN,NaN,4.690000
4,3,44,26.042033,8.2,43.5,-0.024875,-0.292127,49.420879,-1.300360,0.493824,13.131044,0.027065,-1.316498,2.143956,77.109252,7.377355,2.607967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,26,24.221739,17.0,29.9,0.060319,-0.178136,85.498913,-1.928440,4.796944,21.627174,-1.389769,2.260434,6.247826,4.602014,27.896006,3.513043
565,3,43,25.357143,14.4,38.7,0.754646,0.289155,65.911264,-0.283801,-1.338683,18.016209,-1.063751,0.419328,2.120330,33.080004,4.866351,3.153571
566,0,38,25.138816,15.6,34.8,1.004186,1.678955,72.937500,-0.931120,0.947053,19.778947,-1.977677,5.674409,6.615132,2.971615,9.914768,4.592105
567,0,46,29.697561,19.6,42.1,0.298462,-1.048559,58.222764,0.036185,-1.381920,19.526016,-1.020817,1.238418,NaN,NaN,1.238418,3.490244


In [88]:
for column in test.columns:
    index = np.where(test[column].isnull())[0]
    if len(index)!= 0:
        for i in range(len(index)):
            temp = (mean.loc[mean['Crop'] == test['Crop'].loc[index[i]]])
            test[column].loc[index[i]] = temp[column].values
        
        
        
        
        

/usr/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-88-1a3c36829e4f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column].loc[index[i]] = temp[column].values


In [89]:
test

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind
0,1,4,28.131707,19.3,41.0,1.063315,0.476075,69.618699,-0.852685,-0.666886,21.366667,-1.674645,3.143399,2.485778,4.405056,3.143399,2.956911
1,0,23,25.703947,20.0,36.9,1.181141,1.730647,66.792105,-1.093754,1.552880,18.850000,-2.437715,8.002811,6.980263,2.197078,4.908670,4.723684
2,0,29,24.614444,18.9,34.1,0.310620,-0.477274,80.448889,-0.873560,0.647786,21.021111,-0.342050,1.124286,6.210000,9.549663,2.919901,4.145556
3,0,38,25.745556,20.7,41.5,0.538028,-0.982833,77.014444,-1.034927,-0.239949,21.252222,-0.910521,1.303033,6.379201,5.009796,7.454333,4.690000
4,3,44,26.042033,8.2,43.5,-0.024875,-0.292127,49.420879,-1.300360,0.493824,13.131044,0.027065,-1.316498,2.143956,77.109252,7.377355,2.607967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,26,24.221739,17.0,29.9,0.060319,-0.178136,85.498913,-1.928440,4.796944,21.627174,-1.389769,2.260434,6.247826,4.602014,27.896006,3.513043
565,3,43,25.357143,14.4,38.7,0.754646,0.289155,65.911264,-0.283801,-1.338683,18.016209,-1.063751,0.419328,2.120330,33.080004,4.866351,3.153571
566,0,38,25.138816,15.6,34.8,1.004186,1.678955,72.937500,-0.931120,0.947053,19.778947,-1.977677,5.674409,6.615132,2.971615,9.914768,4.592105
567,0,46,29.697561,19.6,42.1,0.298462,-1.048559,58.222764,0.036185,-1.381920,19.526016,-1.020817,1.238418,6.434152,4.970937,1.238418,3.490244


In [90]:
imp = Imputer(missing_values=np.nan, strategy='mean')
imp.fit(test)
test = pd.DataFrame(data=imp.transform(test), columns=test.columns)

In [91]:
test

,Season,Crop,AvgAirTemp,MinAirTemp,MaxAirTemp,AvgTempSkew,AvgTempKurt,AvgRelHum,AvgRelHumSkew,AvgRelHumKurt,AvgDewPt,AvgDewPtSkew,AvgDewPtKurt,AvgPrec,AvgPrecSkew,AvgPrecKurt,AvgWind
0,1.0,4.0,28.131707,19.3,41.0,1.063315,0.476075,69.618699,-0.852685,-0.666886,21.366667,-1.674645,3.143399,2.485778,4.405056,3.143399,2.956911
1,0.0,23.0,25.703947,20.0,36.9,1.181141,1.730647,66.792105,-1.093754,1.552880,18.850000,-2.437715,8.002811,6.980263,2.197078,4.908670,4.723684
2,0.0,29.0,24.614444,18.9,34.1,0.310620,-0.477274,80.448889,-0.873560,0.647786,21.021111,-0.342050,1.124286,6.210000,9.549663,2.919901,4.145556
3,0.0,38.0,25.745556,20.7,41.5,0.538028,-0.982833,77.014444,-1.034927,-0.239949,21.252222,-0.910521,1.303033,6.379201,5.009796,7.454333,4.690000
4,3.0,44.0,26.042033,8.2,43.5,-0.024875,-0.292127,49.420879,-1.300360,0.493824,13.131044,0.027065,-1.316498,2.143956,77.109252,7.377355,2.607967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.0,26.0,24.221739,17.0,29.9,0.060319,-0.178136,85.498913,-1.928440,4.796944,21.627174,-1.389769,2.260434,6.247826,4.602014,27.896006,3.513043
565,3.0,43.0,25.357143,14.4,38.7,0.754646,0.289155,65.911264,-0.283801,-1.338683,18.016209,-1.063751,0.419328,2.120330,33.080004,4.866351,3.153571
566,0.0,38.0,25.138816,15.6,34.8,1.004186,1.678955,72.937500,-0.931120,0.947053,19.778947,-1.977677,5.674409,6.615132,2.971615,9.914768,4.592105
567,0.0,46.0,29.697561,19.6,42.1,0.298462,-1.048559,58.222764,0.036185,-1.381920,19.526016,-1.020817,1.238418,6.434152,4.970937,1.238418,3.490244


In [92]:
X_test = test.values

In [93]:
X_test = x_scaler.transform(X_test)


# RF PRED

In [94]:
y_pred=regr.predict(X_test)

In [95]:
y_pred = y_scaler.inverse_transform(np.asarray(y_pred.reshape(1, -1)))

In [96]:
op = np.concatenate((id_crop.reshape(-1,1), y_pred.reshape(-1,1)), axis=1)

In [97]:
op  =  pd.DataFrame({'ID': op[:, 0], 'Yield': op[:, 1]})

In [98]:
op.ID = op.ID.astype(int)

In [102]:
op.to_csv('rf_pred.csv', index=False)

Predicted negative Yield assigned to Zero.

In [105]:
op.loc[op.Yield<0, 'Yield']=0

In [109]:
op.to_csv('RF_prediction.csv', index=False)

# NN PRED

In [148]:
from keras.models import load_model
model = load_model('crops.hdf5')

In [149]:
y_pred = model.predict(X_test)
y_pred = y_scaler.inverse_transform(np.asarray(y_pred))

In [150]:
op = np.concatenate((id_crop.reshape(-1,1), y_pred.reshape(-1,1)), axis=1)

In [151]:
op  =  pd.DataFrame({'ID': op[:, 0], 'Yield': op[:, 1]})
op.ID = op.ID.astype(int)

In [152]:
op.to_csv('nn_pred.csv', index=False)

Predicted negative Yield assigned to Zero.

In [100]:
op.loc[op.Yield<0, 'Yield']=0

In [110]:
op.to_csv('NN_prediction.csv', index=False)